In [84]:
%pip install --upgrade --quiet  langchain-pinecone langchain-openai langchain langchain_community langchain bs4
%pip install pinecone-client sentence_transformers
%pip install langchainhub

In [85]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import ChatOpenAI



In [86]:
import os
os.environ["PINECONE_API_KEY"] = ""
os.environ['OPENAI_API_KEY'] = ""
index_name = "test-index"


In [87]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [88]:

loader = WebBaseLoader("https://python.langchain.com/v0.1/docs/get_started/introduction")


In [89]:
docs = loader.load()

docs

[Document(metadata={'source': 'https://python.langchain.com/v0.1/docs/get_started/introduction', 'title': 'Introduction | ğŸ¦œï¸�ğŸ”— LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content="\n\n\n\n\nIntroduction | ğŸ¦œï¸�ğŸ”— LangChain\n\n\n\n\n\n\n\nSkip to main contentLangChain v0.2 is out! You are currently viewing the old v0.1 docs. View the latest docs here.ComponentsIntegrationsGuidesAPI ReferenceMorePeopleVersioningContributingTemplatesCookbooksTutorialsYouTubev0.1v0.2v0.1ğŸ¦œï¸�ğŸ”—LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS DocsğŸ’¬SearchGet startedIntroductionQuickstartInstallationUse casesQ&A with RAGExtracting structured outputChatbotsTool use and agentsQuery analysisQ&A over SQL + CSVMoreExpression LanguageGet startedRunnable interfacePrimitivesAdvantages of LCELStreamingAdd message history (memory)MoreEcosystemğŸ¦œğŸ›\xa0ï¸� Lang

In [90]:
embeddings = OpenAIEmbeddings()


In [93]:
from langchain_pinecone import PineconeVectorStore
from langchain_pinecone import PineconeVectorStore




namespace = "my-namespace1"

vectorstore_from_docs = PineconeVectorStore.from_documents(
docs,
index_name=index_name,
embedding=embeddings,
pinecone_api_key=os.environ['PINECONE_API_KEY'],
namespace=namespace,
)





In [96]:
query = "What is langchian"
vectorstore_from_docs.similarity_search(query)

In [97]:
retriever  = vectorstore_from_docs.as_retriever()

In [95]:
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [94]:
rag_chain.invoke("What is langchain")